In [1]:
import pandas as pd
import numpy as np
import datetime

%matplotlib inline

In [2]:
crime_sort = pd.read_hdf('data/crime.h5', 'crime') \
               .set_index('REPORTED_DATE') \
               .sort_index()

In [3]:
crime_sort.index.max()

Timestamp('2017-09-29 06:16:00')

In [4]:
crime_sort = crime_sort[:'2017-8']
crime_sort.index.max()

Timestamp('2017-08-31 23:52:00')

In [5]:
all_data = crime_sort.groupby([pd.Grouper(freq='M'), 'OFFENSE_CATEGORY_ID']).size()
all_data.head()

REPORTED_DATE  OFFENSE_CATEGORY_ID
2012-01-31     aggravated-assault     113
               all-other-crimes       124
               arson                    5
               auto-theft             275
               burglary               343
dtype: int64

In [6]:
all_data = all_data.sort_values().reset_index(name='Total')
all_data.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total
0,2014-12-31,murder,1
1,2013-01-31,arson,1
2,2016-05-31,murder,1
3,2012-12-31,murder,1
4,2016-12-31,murder,1


In [7]:
goal = all_data[all_data['REPORTED_DATE'] == '2017-8-31'].reset_index(drop=True)
goal['Total_Goal'] = goal['Total'].mul(.8).astype(int)
goal.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total,Total_Goal
0,2017-08-31,murder,7,5
1,2017-08-31,arson,7,5
2,2017-08-31,sexual-assault,57,45
3,2017-08-31,robbery,108,86
4,2017-08-31,white-collar-crime,138,110


In [8]:
pd.merge_asof(goal, all_data, left_on='Total_Goal', right_on='Total', 
              by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last'))

MergeError: incompatible merge keys, must be the same type

In [10]:
pd.merge_sort(goal, all_data, left_on='Total_Goal', right_on='Total', 
              by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last'))

AttributeError: module 'pandas' has no attribute 'merge_sort'

In [11]:
pd.merge_asof(goal, all_data, left_on='Total_Goal', right_on='Total', 
              by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last'))

MergeError: incompatible merge keys, must be the same type

In [12]:
pd.Period(year=2012, month=5, day=17, hour=14, minute=20, freq='T')

Period('2012-05-17 14:20', 'T')

In [13]:
crime_sort.index.to_period('M')

PeriodIndex(['2012-01', '2012-01', '2012-01', '2012-01', '2012-01', '2012-01',
             '2012-01', '2012-01', '2012-01', '2012-01',
             ...
             '2017-08', '2017-08', '2017-08', '2017-08', '2017-08', '2017-08',
             '2017-08', '2017-08', '2017-08', '2017-08'],
            dtype='period[M]', name='REPORTED_DATE', length=453568, freq='M')

In [14]:
ad_period = crime_sort.groupby([lambda x: x.to_period('M'), 
                                'OFFENSE_CATEGORY_ID']).size()
ad_period = ad_period.sort_values() \
                     .reset_index(name='Total') \
                     .rename(columns={'level_0':'REPORTED_DATE'})
ad_period.head()

,REPORTED_DATE,OFFENSE_CATEGORY_ID,Total
0,2014-12,murder,1
1,2013-01,arson,1
2,2016-05,murder,1
3,2012-12,murder,1
4,2016-12,murder,1


In [15]:
cols = ['OFFENSE_CATEGORY_ID', 'Total']
all_data[cols].equals(ad_period[cols])

True

In [16]:
aug_2018 = pd.Period('2017-8', freq='M')
goal_period = ad_period[ad_period['REPORTED_DATE'] == aug_2018].reset_index(drop=True)
goal_period['Total_Goal'] = goal_period['Total'].mul(.8).astype(int)

pd.merge_asof(goal_period, ad_period, left_on='Total_Goal', right_on='Total', 
                  by='OFFENSE_CATEGORY_ID', suffixes=('_Current', '_Last')).head()

MergeError: incompatible merge keys, must be the same type